In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt

In [2]:
test_transaction = dt.fread("/home/rmodi/Documents/CSE519/hw2/dataset/test_transaction.csv").to_pandas()
test_identity = dt.fread("/home/rmodi/Documents/CSE519/hw2/dataset/test_identity.csv").to_pandas()

testset = pd.merge(test_transaction, test_identity, on='TransactionID', how='outer')
%reset_selective -f test_transaction
%reset_selective -f test_identity

testset = testset[["TransactionID","DeviceType","DeviceInfo","TransactionDT","TransactionAmt","ProductCD","card4","card6","P_emaildomain","R_emaildomain","addr1","addr2","dist1","dist2"]]

testset['TransactionDT_day'] = testset['TransactionDT'].apply(lambda x: int(x/86400))
testset['TransactionDT_hour'] = testset['TransactionDT'].apply(lambda x: int(x/86400%1*24))
testset['TransactionDT_min'] = testset['TransactionDT'].apply(lambda x: int(x/86400%1*24%1*60))
testset['TransactionDT_sec'] = testset['TransactionDT'].apply(lambda x: int(x/86400%1*24%1*60%1*60))

In [3]:
# Create a copy of trainset for easy resetting
dataset = testset.copy()

In [4]:
# Replace columns that have empty value with 'unknown' value
cols = ['card4', 'card6', 'DeviceType', 'DeviceInfo', 'P_emaildomain', 'R_emaildomain']
dataset[cols] = dataset[cols].replace({'': 'unknown'})

# Keep the top 5 column values and group remaining ones into 'Others'
top5_deviceinfo = set(dataset['DeviceInfo'].value_counts()[:5].index)
dataset['DeviceInfo'] = dataset['DeviceInfo'].apply(lambda x: x if x in top5_deviceinfo else 'Others')

# Group categories that are similar into one category [4]
regex_patterns = {
    r'^frontier.*$': 'frontier.com',
    r'^gmail.*$': 'gmail.com',
    r'^hotmail.*$': 'hotmail.com',
    r'^live.*$': 'live.com',
    r'^netzero.*$': 'netzero.com',
    r'^outlook.*$': 'outlook.com',
    r'^yahoo.*$': 'yahoo.com'
}
replacements = {
    'P_emaildomain': regex_patterns,
    'R_emaildomain': regex_patterns
}

dataset.replace(replacements, regex=True, inplace=True)

dataset['hr_sin'] = np.sin((dataset['TransactionDT_hour'] + dataset['TransactionDT_min']/60.0)*(np.pi/12.0))
dataset['hr_cos'] = np.cos((dataset['TransactionDT_hour'] + dataset['TransactionDT_min']/60.0)*(np.pi/12.0))
dataset['TransactionAmt_lg'] = np.log(dataset['TransactionAmt'])

from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()
dataset['TransactionAmt_scaled'] = rob_scaler.fit_transform(dataset['TransactionAmt'].values.reshape(-1,1))

categorical_cols = ['DeviceType', 'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain']
ohe = pd.get_dummies(dataset[categorical_cols])
dataset = dataset.join(ohe)
dataset['card6_debit or credit'] = 0.0

train_cols = ["TransactionAmt_scaled","hr_sin","hr_cos"] + list(ohe.columns)

In [5]:
import joblib
clf = joblib.load('rs_rus_xgb.pkl')
test_preds_proba = clf.predict_proba(dataset[train_cols])
test_preds = clf.predict(dataset[train_cols])

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109'] ['TransactionAmt_scaled', 'hr_sin', 'hr_cos', 'DeviceType_desktop', 'DeviceType_mobile', 'DeviceType_unknown', 'ProductCD_C', 'ProductCD_H', 'ProductCD_R', 'ProductCD_S', 'ProductCD_W', 'card4_

In [39]:
pd.DataFrame(test_preds)[0].value_counts()

False    357075
True     149616
Name: 0, dtype: int64

In [21]:
output = pd.DataFrame({'TransactionID': testset.TransactionID, 'isFraud': test_preds_proba[:,1]})

In [22]:
output.to_csv('dataset/submission_4.csv', index=False)

In [50]:
train_cols

['TransactionAmt_scaled',
 'hr_sin',
 'hr_cos',
 'DeviceType_desktop',
 'DeviceType_mobile',
 'DeviceType_unknown',
 'ProductCD_C',
 'ProductCD_H',
 'ProductCD_R',
 'ProductCD_S',
 'ProductCD_W',
 'card4_american express',
 'card4_discover',
 'card4_mastercard',
 'card4_unknown',
 'card4_visa',
 'card6_charge card',
 'card6_credit',
 'card6_debit',
 'card6_debit or credit',
 'card6_unknown',
 'P_emaildomain_aim.com',
 'P_emaildomain_anonymous.com',
 'P_emaildomain_aol.com',
 'P_emaildomain_att.net',
 'P_emaildomain_bellsouth.net',
 'P_emaildomain_cableone.net',
 'P_emaildomain_centurylink.net',
 'P_emaildomain_cfl.rr.com',
 'P_emaildomain_charter.net',
 'P_emaildomain_comcast.net',
 'P_emaildomain_cox.net',
 'P_emaildomain_earthlink.net',
 'P_emaildomain_embarqmail.com',
 'P_emaildomain_frontier.com',
 'P_emaildomain_gmail.com',
 'P_emaildomain_gmx.de',
 'P_emaildomain_hotmail.com',
 'P_emaildomain_icloud.com',
 'P_emaildomain_juno.com',
 'P_emaildomain_live.com',
 'P_emaildomain_mac.c